In [2]:
# Script for multiple runs of wanderlust 
%load_ext autoreload
%autoreload 2

import warnings

# data processing
with warnings.catch_warnings():
    warnings.simplefilter('ignore', FutureWarning)
    import scanpy.api as sc
import numpy as np
import scanpy.api as sc
import pandas as pd
import os
import scipy.sparse as sp

# plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('white')
matplotlib.rcParams['figure.figsize'] = [4, 4]
matplotlib.rcParams['figure.dpi'] = 100
warnings.filterwarnings(action="ignore", module="matplotlib", message="findfont")

# change logging settings for scanpy
sc.settings.verbosity = 4  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures
sc.settings.n_jobs = 30
sc.logging.print_version_and_date()
sc.logging.print_versions_dependencies_numerics()

Running Scanpy 0+unknown on 2018-04-10 21:39.
Dependencies: anndata==0.5.8 numpy==1.14.2 scipy==1.0.1 pandas==0.22.0 scikit-learn==0.19.1 statsmodels==0.8.0 


In [3]:
from sklearn.metrics import pairwise_distances
def _find_cell_indices(adata, cells):
    """Function to find cell indices given index in the AnnData object
    """
    return np.where(adata.obs_names.isin(cells))[0]
        
class SamplingDistanceEstimator:

    def __init__(self, adata: sc.AnnData, ref_set_sigmas: pd.Series, ):
        """Class the computes the distance for the subsample. The score determines the 
        fraction of reference spheres that are unoccupied
        
        :param sc.AnnData adata: Scanpy AnnData object containing the normalized count matrix
        :param pd.Series ref_set_sigmas: Pandas series representing the radii of the 
        reference set
        """
        
        if any(~ref_set_sigmas.index.isin(adata.obs_names)):
            raise ValueError(
                'Some of the cells in the reference set are not in the AnnData object. '
                'Ensure that all the reference cells are in the AnnData object'
            )
        
        self.adata = adata
        self.ref_set = _find_cell_indices(adata, ref_set_sigmas.index)
        self.sigmas = ref_set_sigmas[adata.obs_names[self.ref_set]].values
        
        
    def determine_ref_occupancy(self, test_set: pd.Index, block_size=7500, n_jobs=1):
        """ Function to determine the number of test cells occupying each
        reference sphere
        
        :param pd.Index test_set: Pandas index of test set observation names
        """
        if any(~test_set.isin(adata.obs_names)):
            raise ValueError(
                'Some of the cells in the test set are not in the AnnData object. '
                'Ensure that all the test cells are in the AnnData object'
            )
        
        # Test data
        test_data = self.adata[_find_cell_indices(self.adata, test_set),].X
            
        # Compute counts in blocks
        counts = np.zeros(len(self.ref_set))
        blocks = np.linspace(0, len(self.ref_set), 
                             int(len(self.ref_set)/block_size)+1).astype(np.int)
        for b in range(1, len(blocks)):
            test_range = range(blocks[b-1], blocks[b])
            ref_data = self.adata[self.ref_set[test_range],].X
            dists = pairwise_distances(ref_data, test_data, n_jobs=n_jobs)
            counts[test_range] = (dists < self.sigmas[test_range, np.newaxis]).sum(axis=1)
            
        return counts

    
    def determine_distance(self, test_set: pd.Index):
        """ Function to determine the fraction of unoccoupied reference 
        spheres in the test set
        
        :param pd.Index test_set: Pandas index of test set observation names
        """
        if any(~test_set.isin(adata.obs_names)):
            raise ValueError(
                'Some of the cells in the test set are not in the AnnData object. '
                'Ensure that all the test cells are in the AnnData object'
            )
        counts = self.determine_ref_occupancy(test_set)
        return np.sum(counts == 0)/len(counts), counts
        
    def determine_distance_from_occupancy(self, ref_sphere_counts):
        """ Function to determine distance from ref spehere counts
        
        :param pd.Index test_set: Pandas index of test set observation names
        """
        return np.sum(ref_sphere_counts==0)/len(ref_sphere_counts)
        

In [4]:
adata = sc.read_h5ad('../../data/tasks/sampling/ica_bone_marrow.h5ad')

In [5]:
ref_set_sigmas = pd.read_csv('../../data/tasks/sampling/ica_bone_marrow_ref_set_sigmas.csv', index_col=0, header=None).iloc[:, 0]

In [13]:
def get_magic_number(file):
    udata = sc.read(file)
    our_set_=udata.obs_names
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sde = SamplingDistanceEstimator(adata, ref_set_sigmas)
    dist, init_ref_sphere_counts = sde.determine_distance(our_set_)
    print(file,dist)
    return dist,init_ref_sphere_counts

In [21]:
file='./write/samp_data_v1.h5ad'

In [22]:
%%time
dist,init_ref_sphere_counts=get_magic_number(file)

    normalizing by total count per cell
    filtered out 0 cells that have less than 1 counts
        finished (0:00:04.97): normalized adata.X and added
        'n_counts', counts per cell before normalization (adata.obs)
./write/samp_data_v1.h5ad 0.3738666666666667
CPU times: user 5min 1s, sys: 3.12 s, total: 5min 4s
Wall time: 5min 4s


In [23]:
file='./write/samp_data_v1_small.h5ad'

In [24]:
%%time
dist,init_ref_sphere_counts=get_magic_number(file)

    normalizing by total count per cell
    filtered out 0 cells that have less than 1 counts
        finished (0:00:04.95): normalized adata.X and added
        'n_counts', counts per cell before normalization (adata.obs)
./write/samp_data_v1_small.h5ad 0.5168666666666667
CPU times: user 2min 10s, sys: 1.84 s, total: 2min 12s
Wall time: 2min 12s


In [19]:
file='./write/samp_data_v1_baseline_20k.h5ad'

In [20]:
%%time
dist,init_ref_sphere_counts=get_magic_number(file)

    normalizing by total count per cell
    filtered out 0 cells that have less than 1 counts
        finished (0:00:04.95): normalized adata.X and added
        'n_counts', counts per cell before normalization (adata.obs)
./write/samp_data_v1_baseline_20k.h5ad 0.3712
CPU times: user 3min 36s, sys: 2.59 s, total: 3min 39s
Wall time: 3min 39s


In [17]:
file='./write/samp_data_v1_baseline_5k.h5ad'

In [18]:
%%time
dist,init_ref_sphere_counts=get_magic_number(file)

    normalizing by total count per cell
    filtered out 0 cells that have less than 1 counts
        finished (0:00:04.96): normalized adata.X and added
        'n_counts', counts per cell before normalization (adata.obs)
./write/samp_data_v1_baseline_5k.h5ad 0.5971333333333333
CPU times: user 1min 52s, sys: 1.85 s, total: 1min 54s
Wall time: 1min 54s


In [15]:
file='./write/samp_data_v1_baseline_100.h5ad'

In [16]:
%%time
dist,init_ref_sphere_counts=get_magic_number(file)

    normalizing by total count per cell
    filtered out 0 cells that have less than 1 counts
        finished (0:00:04.96): normalized adata.X and added
        'n_counts', counts per cell before normalization (adata.obs)
./write/samp_data_v1_baseline_100.h5ad 0.6972
CPU times: user 1min 16s, sys: 1.66 s, total: 1min 17s
Wall time: 1min 17s
